http://anie.me/On-Torchtext/

In [1]:
from torchtext import data

In [2]:
class Arguments():
    def __init__(self, batch_size):
        self.batch_size = batch_size
        
args = Arguments(32)

In [3]:
TEXT = data.Field(batch_first=True,tokenize="spacy") #by default tokenize simply splits on spaces, this uses the spaCy tokenizer, but must be installed via `conda install spacy`
LABELS = data.Field(batch_first=True,pad_token=None, unk_token=None) #don't want the <pad> or <unk> tokens in the labels



    Only loading the 'en' tokenizer.



In [4]:
train, dev, test = data.TabularDataset.splits(
    path='data/processed/', train='train.txt',
    validation='dev.txt', test='test.txt', format='tsv',
    fields=[('sentence1', TEXT), ('sentence2', TEXT), ('labels', LABELS)])

In [5]:
print(train.fields)

{'sentence1': <torchtext.data.field.Field object at 0x7fdc29c67780>, 'sentence2': <torchtext.data.field.Field object at 0x7fdc29c67780>, 'labels': <torchtext.data.field.Field object at 0x7fdc2903dc88>}


In [6]:
print(len(train))

549367


In [7]:
print(vars(train[0]))

{'sentence1': ['A', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.'], 'sentence2': ['A', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.'], 'labels': ['neutral']}


In [15]:
train_iter, dev_iter, test_iter = data.Iterator.splits((train, dev, test), #the dataset object to load data from
                                                            batch_size = args.batch_size, 
                                                            #sort_key=lambda x: len(x.sentence1), #how should it organise the buckets?
                                                            device=-1) #-1 implies using the CPU

In [16]:
TEXT.build_vocab(train.sentence1)
TEXT.build_vocab(train.sentence2)
LABELS.build_vocab(train.labels)

In [17]:
print(len(TEXT.vocab))

35080


In [18]:
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             '.': 2,
             'a': 3,
             'is': 4,
             'A': 5,
             'the': 6,
             'The': 7,
             'are': 8,
             'man': 9,
             'in': 10,
             'on': 11,
             'woman': 12,
             'of': 13,
             'to': 14,
             'people': 15,
             'at': 16,
             'and': 17,
             'Two': 18,
             'with': 19,
             'his': 20,
             'playing': 21,
             'girl': 22,
             'for': 23,
             'men': 24,
             'boy': 25,
             'dog': 26,
             'There': 27,
             'outside': 28,
             'wearing': 29,
             'person': 30,
             'sitting': 31,
             'her': 32,
             'group': 33,
             'an': 34,
             'People': 35,
             'child': 36,
             'women': 37,
           

In [19]:
print(LABELS.vocab.itos)

['entailment', 'contradiction', 'neutral']


In [20]:
batch = next(iter(train_iter))
print(batch.sentence1)
print(batch.sentence2)
print(batch.labels)

Variable containing:

Columns 0 to 10 
    43     55     17     65     56    340     10      6    140      1      1
     5   1924   5123   4240   7480   7585   3318     26   5945    672    107
  4705     13  15586    188   1755     96     14    926      6   2452    307
     5     33     13     12   1015      6   1723     87    161    563     23
     5     85     22     10      3     65    202     54     83      3    424
     5      9     52      3    327     73      1      1      1      1      1
     5     26    236     10      6    213     14    325      3     72     80
     5      9     10      3     72     58     17   1236    567    116      2
  6655     10     68    477     42      3    837    221      2      1      1
     5     33     13     15      8    439     14    266    556      2      1
     5      9     29   2412   2622    257      4     54     83      3   1287
     5     22     29      3    253   1803    184     83     32    187      2
    27      4      3    287  19530  1